# <b><span style="color:#D65DB1"> Use Decision Tree Classifier to give score to the transcribed results </span></b>

The innput will be the CER score, and the output is the number 1 to 5.
The Clasefier is a Decision Tree from sklearn library.

Implementation:
1)  Get the CER from the assesment file. 
    This is calculated using pronScores.
2)  Make a new df wher the assesment 'CER' and global 'Score' is.
3)  Split the df into traing and testing data.
4)  Initilize a Decision Tree Classifier
5)  Train the classefier, and test it.
6)  Opimize the model if needed
7)  Change the predicted Score in the transcribed files and update the CSV and results.

For 'Score' 5 where 'CER' is more than 0.0, is one incstand at index 7115. With the CER score of 0.333333.

<!-- 
# Code
filtered_df = input_assesmen[(input_assesmen['Score'] == 5) & (input_assesmen['CER'] > 0)]
print("\nFiltered rows where 'Score' is 5 and 'CER' is greater than 0:\n", filtered_df) 
->

<!-- 
# Test for forståelse
# df_assesment.loc[0]
# print(len(df_assesment['Pronunciation'].loc[0].split(' ')))
# print(len(df_assesment['pronScores'].loc[0].split(' ')))
# pronScore =  df_assesment['pronScores'].loc[0].split(' ')
# CER = pronScore.count('0')/len(df_assesment['pronScores'].loc[0].split(' '))

# print(CER)
# print(pronScore)
# print(pronScore.count('0'))
# print(pronScore.count('1'))
# # Double check that the len of the pronScore is the same as the len of the Pronunciation
# for idx, row in df_assesment.iterrows():
#     pron = len(row['Pronunciation'].split(' '))
#     pronScor = len(row['pronScores'].split(' '))
#     if pron != pronScor:
#         print('index', idx)
#         print('Pronunciation', pron)
#         print('pronScores', pronScor) 
-->


Colors: 

<b><span style="color:#7ABCDD">7ABCDD</span></b>
<b><span style="color:#57D0E2">57D0E2</span></b>
<b><span style="color:#4DE1D4">4DE1D4</span></b>
<b><span style="color:#77EFB6">77EFB6</span></b>
<b><span style="color:#B4F790">B4F790</span></b>
<b><span style="color:#F9F871">F9F871</span></b>

<b><span style="color:#D4A5CA">D4A5CA</span></b>
<b><span style="color:#FBA8C2">FBA8C2</span></b>
<b><span style="color:#FFAFAC">FFAFAC</span></b>
<b><span style="color:#FFC08F">FFC08F</span></b>
<b><span style="color:#FFDA77">FFDA77</span></b>
<b><span style="color:#F9F871">F9F871</span></b>

<b><span style="color:#845EC2">845EC2</span></b>
<b><span style="color:#D65DB1">D65DB1</span></b>
<b><span style="color:#FF6F91">FF6F91</span></b>
<b><span style="color:#FF9671">FF9671</span></b>
<b><span style="color:#FFC75F">FFC75F</span></b>
<b><span style="color:#F9F871">F9F871</span></b>








<b><span style="color:#845EC2">845EC2</span></b>
<b><span style="color:#2C73D2">2C73D2</span></b>
<b><span style="color:#0081CF">0081CF</span></b>
<b><span style="color:#0089BA">0089BA</span></b>
<b><span style="color:#008E9B">008E9B</span></b>
<b><span style="color:#008F7A">008F7A</span></b>

## <span style="color:#FF6F91"> Problems whit the current methode: </span> 

The dataset is <b>unbalanced</b>. 
Instances of 'Score' 4 is higest for the same group as 'Score' 5. 

'Score' 4 is never the highest number for each of the CER scores, so it will not be used in the classefiing process. 

This means in the reality that <b>the model will only be able to predict 'Score' 1, 2, 3 and 5</b>, where most of them is the lower scores.

<span style="color:#FF9671"><b> Try: micro-averaging </b></span>

In [4]:
import self_made_functions as smf 
import pandas as pd

df_assesment, wv_path = smf.get_correct_df()

# # File name	Score	Prosody	Noise/Disruption	Pre-speech noise	Repetition	Word	Pronunciation	pronScores	Assessor
csv_df  = pd.read_csv('Transcriptions/transcriptions_nb-whisper-medium-verbatim_v1.csv') # Test for 1 file first

In [9]:
# make al values in cloumn = NaN
csv_df["CER Score"] = float('nan')
# Add ID column
csv_df["ID"] = csv_df["File name"].apply(lambda x: x.split('_')[0])
# Add CER Score for original score
csv_df["og_cer_score"] = df_assesment["pronScores"].apply(lambda x: x.split(' ').count('1')/len(x.split(' ')))

# To ulike resultater -> hvor godt matcher CER opp mot OG CER
# 2. hvor godt matcher CER/ordet opp mot selve target ordet

csv_df

,File name,Word,Transcribed,CER (Character Error Rate),CER Score,OG Score,CER Output,ID,og_cer_score
0,a06_hylle.wav,hylle,hyll,0.250000,NaN,1,"CharacterOutput(references=[['h', 'y', 'l', 'l...",a06,0.800000
1,a12_hylle.wav,hylle,hylle,0.000000,NaN,5,"CharacterOutput(references=[['h', 'y', 'l', 'l...",a12,0.666667
2,a33_hylle.wav,hylle,hylle,0.000000,NaN,5,"CharacterOutput(references=[['h', 'y', 'l', 'l...",a33,0.666667
3,d18_hylle.wav,hylle,hylle,0.000000,NaN,5,"CharacterOutput(references=[['h', 'y', 'l', 'l...",d18,0.500000
4,a27_hylle.wav,hylle,kylling,0.571429,NaN,3,"CharacterOutput(references=[['k', 'y', 'l', 'l...",a27,0.666667
...,...,...,...,...,...,...,...,...,...
9311,d16_smart.wav,smart,snart,0.200000,NaN,4,"CharacterOutput(references=[['s', 'n', 'a', 'r...",d16,1.000000
9312,a10_smart.wav,smart,schmach,0.571429,NaN,2,"CharacterOutput(references=[['s', 'c', 'h', 'm...",a10,0.666667
9313,a04_smart.wav,smart,smart,0.000000,NaN,4,"CharacterOutput(references=[['s', 'm', 'a', 'r...",a04,1.000000
9314,a25_smart.wav,smart,snart,0.200000,NaN,4,"CharacterOutput(references=[['s', 'n', 'a', 'r...",a25,0.800000


In [17]:
from transformers import pipeline
model_name_path = 'NbAiLabBeta/nb-whisper-base'
model = pipeline("automatic-speech-recognition", model_name_path) # Load the model

# From Phind.com
from transformers import WhisperTokenizer, WhisperForConditionalGeneration
import soundfile as sf
import os
name = 'd17_loeve.wav'
# Load an audio file
audio_file = os.path.join(wv_path,name)

transcription = model(audio_file, generate_kwargs={'task': 'transcribe', 'language': 'no'}) 

# Load the Whisper model and tokenizer
tokenizer = WhisperTokenizer.from_pretrained(model_name_path)
model = WhisperForConditionalGeneration.from_pretrained(model_name_path)

# audio, sample_rate = sf.read(audio_file)

# # Preprocess the audio
input_values = tokenizer(transcription['text'], return_tensors="pt").input_values

# # Generate a prediction
with torch.no_grad():
    logits = model(input_values).logits

# # Decode the logits to text
# predicted_ids = torch.argmax(logits, dim=-1)
# transcription = tokenizer.decode(predicted_ids[0])

# print(transcription)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


AttributeError: 

In [5]:
import torch
import soundfile as sf
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# Initialize the WhisperProcessor
processor = WhisperProcessor.from_pretrained("openai/whisper-base")

# Load an audio file
audio_file = os.path.join(wv_path,name)
audio, sample_rate = sf.read(audio_file)

# Preprocess the audio
input_values = processor(audio, sampling_rate=sample_rate, return_tensors="pt").input_values

# Load the Whisper model
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")

# Generate a prediction
with torch.no_grad():
    logits = model(input_values).logits

# Decode the logits to text
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.decode(predicted_ids[0])

print(transcription)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


AttributeError: 

In [132]:
# Make the input used for train the classefier model
input_assesmen = pd.DataFrame(columns=['CER', 'Score'])

same_score = pd.DataFrame(columns=['pronScores', 'Score'])

for idx, row in df_assesment.iterrows():
    pronScore = row['pronScores'].split(' ')
    CER = pronScore.count('0')/len(pronScore)
    
    one = pronScore.count('1')
    values_one = {
        'pronScore': one,
        'Score': row['Score']
    }
    values_one = pd.DataFrame(values_one, index=[0])
    same_score = pd.concat([same_score, values_one], ignore_index=True)
    
    
    values = {
            'CER': CER,
            'Score': row['Score']
        }
    values = pd.DataFrame(values, index=[0])
    # if values.empty: No emty enteries
    #     print('empty', idx)
    # elif input_assesmen.empty:
    #     print('empty', idx)
        
    input_assesmen = pd.concat([input_assesmen, values], ignore_index=True)

/tmp/ipykernel_3133737/483509519.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  input_assesmen = pd.concat([input_assesmen, values], ignore_index=True)


In [143]:
# Some of the scores has the same CER value, so this should be filered out before training the model

input_group = input_assesmen.groupby('CER')

for idx, group in input_group:
    print('CER            ', group['CER'].unique())
    print('Scores in group', group['Score'].unique())
    print('Number of each score in group', group['Score'].value_counts())
    print()

CER             [0.]
Scores in group [5 4 3]
Number of each score in group Score
5    3277
4    1064
3       1
Name: count, dtype: int64

CER             [0.14285714]
Scores in group [3 4]
Number of each score in group Score
3    80
4    35
Name: count, dtype: int64

CER             [0.16666667]
Scores in group [3 4]
Number of each score in group Score
3    171
4     62
Name: count, dtype: int64

CER             [0.2]
Scores in group [4 3]
Number of each score in group Score
3    341
4    224
Name: count, dtype: int64

CER             [0.25]
Scores in group [3 4 2]
Number of each score in group Score
3    676
4    454
2      1
Name: count, dtype: int64

CER             [0.28571429]
Scores in group [3 2]
Number of each score in group Score
3    34
2    16
Name: count, dtype: int64

CER             [0.33333333]
Scores in group [3 4 2 1 5]
Number of each score in group Score
3    587
4    472
2     58
1      1
5      1
Name: count, dtype: int64

CER             [0.4]
Scores in group [2 3 

In [128]:
values_one[values_one['Score']].CER.value_counts()




In [119]:
# print(input_assesmen)
# # NB! some of CER for 3 == CER for 4
# # Check how many of the scores has the same CER
# print(input_assesmen[input_assesmen['Score'] == 3].CER.value_counts())
# print(input_assesmen[input_assesmen['Score'] == 4].CER.value_counts())
# print(input_assesmen[input_assesmen['Score'] == 5].CER.value_counts())

In [120]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
#   PROBLEM!!!!!!!!!!!!!!!! IT DOES NOT TRAIN ON NR 4 FOR SOME REASON! WHY?!

# Split data into X (features) and y (labels)
X = input_assesmen[['CER']]
y = input_assesmen['Score'].astype(int) 

# Split data
# Set random state for reproducibility (same results every time we run the code)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train
clf = DecisionTreeClassifier(random_state=42) 
clf.fit(X_train, y_train)

# Make prediction
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           1       0.35      0.55      0.43        22
           2       0.83      0.23      0.35       173
           3       0.58      0.98      0.73       550
           4       0.00      0.00      0.00       487
           5       0.75      1.00      0.85       633

    accuracy                           0.66      1865
   macro avg       0.50      0.55      0.47      1865
weighted avg       0.50      0.66      0.54      1865



/home/ajtruyen/anaconda3/envs/master_amanda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ajtruyen/anaconda3/envs/master_amanda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ajtruyen/anaconda3/envs/master_amanda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [121]:
# Train the model using GridSearch
from sklearn.model_selection import GridSearchCV
param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}


# Initialize DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=42)
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Print best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

# Use the best estimator for predictions
best_clf = grid_search.best_estimator_
y_pred = best_clf.predict(X_test)

# Evaluate the classifier
print(classification_report(y_test, y_pred))

Best Parameters: {'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2}
Best Score: 0.666086060599522
              precision    recall  f1-score   support

           1       0.33      0.50      0.40        22
           2       0.81      0.23      0.35       173
           3       0.58      0.98      0.73       550
           4       0.00      0.00      0.00       487
           5       0.75      1.00      0.85       633

    accuracy                           0.66      1865
   macro avg       0.49      0.54      0.47      1865
weighted avg       0.50      0.66      0.54      1865



/home/ajtruyen/anaconda3/envs/master_amanda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ajtruyen/anaconda3/envs/master_amanda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ajtruyen/anaconda3/envs/master_amanda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [ ]:

# # Example prediction for a new data point
# new_CER = [[your_new_CER_value]]  # replace with actual new CER value
# predicted_score = clf.predict(new_CER)
# print(f"Predicted score for CER {new_CER}: {predicted_score}")
# # 

In [ ]:
# Put specific sound files in a new direcoty:

new_dir = 'SoundFiles_I_want_to_liten_to'


# Kalkuler CER for Assesments og kun bruk cer mot pronScore

# df_assesment[df_assesment['File name']=='a01_barn.wav']

# total_char = 

# CER= 
# Total Number of Characters in the Reference
# Number of Insertions+Number of Deletions+Number of Substitutions
# ​


NameError: name 'SoundFiles_I_want_to_liten_to' is not defined